In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('../data/train.csv')
df.head()

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00


In [3]:
df.describe(include='object')

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,37d61fd2272659b1,component composite coating,composition,H01
freq,1,152,24,2186


In [4]:
df['input'] = 'TEXT1: '+ df.context + '; TEXT2: ' + df.target + '; ANC1: ' + df.anchor

In [5]:
df['input'].head()

0    TEXT1: A47; TEXT2: abatement of pollution; ANC...
1    TEXT1: A47; TEXT2: act of abating; ANC1: abate...
2    TEXT1: A47; TEXT2: active catalyst; ANC1: abat...
3    TEXT1: A47; TEXT2: eliminating process; ANC1: ...
4    TEXT1: A47; TEXT2: forest region; ANC1: abatement
Name: input, dtype: object

In [6]:
from datasets import Dataset, DatasetDict
# Tokenization: split each text up into words (tokens)
# Numericalization: convert each word (token) into number
# Transformers uses Dataset object for storing dataset! We can 
# create one as follows
ds = Dataset.from_pandas(df)


In [7]:
model_nm = 'microsoft/deberta-v3-small'

In [8]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
tokz = AutoTokenizer.from_pretrained(model_nm) # get the preprocessing 

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/opt/conda/lib/python3.7/site-packages/transformers/convert_slow_tokenizer.py:447: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  "The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option"
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [9]:
tokz.tokenize('TEXT1: A47; TEXT2: abatement of pollution; ANC1: abatement')

['▁TEXT',
 '1',
 ':',
 '▁A',
 '47',
 ';',
 '▁TEXT',
 '2',
 ':',
 '▁abatement',
 '▁of',
 '▁pollution',
 ';',
 '▁ANC',
 '1',
 ':',
 '▁abatement']

In [10]:
# turn token to number
def tok_func(x):
  return tokz(x['input']) # use the same preprocessing on out input data

In [11]:
tok_ds= ds.map(tok_func, batched=True)

  0%|          | 0/37 [00:00<?, ?ba/s]

In [12]:
row = tok_ds[0]
row['input'], row['input_ids']

('TEXT1: A47; TEXT2: abatement of pollution; ANC1: abatement',
 [1,
  54453,
  435,
  294,
  336,
  5753,
  346,
  54453,
  445,
  294,
  47284,
  265,
  6435,
  346,
  23702,
  435,
  294,
  47284,
  2])

In [13]:
tokz.vocab['▁of']

265

In [14]:
tok_ds = tok_ds.rename_columns({'score':'labels'})

In [15]:
# Test and validation sets
eval_df = pd.read_csv('../data/test.csv')
eval_df.describe()

,id,anchor,target,context
count,36,36,36,36
unique,36,34,36,29
top,4112d61851461f60,el display,inorganic photoconductor drum,G02
freq,1,2,1,3


In [16]:
# Transformers uses DatasetDict for holding training and validation set
# To create one that contain 25% validation and 75% training set user train_test_splt on Dataset
dds = tok_ds.train_test_split(0.25, seed=42)

In [17]:
# create metric
import numpy as np

def corr(x, y):
  return np.corrcoef(x, y)[0][1]

def corr_d(eval_pred):
  return {'pearson':corr(*eval_pred)}

In [18]:
# Training our model
# To train a model in transformers we'll need this:
from transformers import TrainingArguments, Trainer

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The 

In [19]:
bs = 128
epochs = 4
lr = 8e-5

In [20]:
args = TrainingArguments(
    output_dir='outputs',
    learning_rate=lr,
    warmup_ratio=0.1,
    lr_scheduler_type='cosine',
    fp16=True, # if cuda, set it to True
    evaluation_strategy='epoch',
    per_device_train_batch_size=bs,
    per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs,
    weight_decay=0.01,
    report_to='none'
)


In [21]:
# Create model and trainer which combines the data and model together
model = AutoModelForSequenceClassification.from_pretrained(model_nm, num_labels=1)
trainer = Trainer(
    model,
    args,
    train_dataset=dds['train'],
    eval_dataset=dds['test'],
    tokenizer=tokz,
    compute_metrics=corr_d
)

Some weights of the model checkpoint at microsoft/deberta-v3-small were not used when initializing DebertaV2ForSequenceClassification: ['mask_predictions.LayerNorm.weight', 'mask_predictions.dense.bias', 'mask_predictions.dense.weight', 'mask_predictions.classifier.weight', 'lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.bias', 'mask_predictions.LayerNorm.bias', 'lm_predictions.lm_head.dense.bias', 'mask_predictions.classifier.bias', 'lm_predictions.lm_head.LayerNorm.weight']
- This IS expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaV2ForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from 

In [22]:
trainer.train()

/opt/conda/lib/python3.7/site-packages/transformers/optimization.py:395: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Pearson
1,No log,0.028622,0.801928
2,No log,0.024775,0.826087
3,0.039000,0.022813,0.834306
4,0.039000,0.022503,0.835362


TrainOutput(global_step=856, training_loss=0.028351240068952614, metrics={'train_runtime': 270.9169, 'train_samples_per_second': 403.873, 'train_steps_per_second': 3.16, 'total_flos': 716605488222960.0, 'train_loss': 0.028351240068952614, 'epoch': 4.0})

In [40]:
# Transform eval_df so tokens the model can take as input
eval_df['input'] = 'TEXT1: '+ eval_df.context + '; TEXT2: ' + eval_df.target + '; ANC1: ' + eval_df.anchor
eval_ds = Dataset.from_pandas(eval_df)
eval_ds = eval_ds.map(tok_func, batched=True)

  0%|          | 0/1 [00:00<?, ?ba/s]

In [42]:
# Do prediction on eval_ds
pred = trainer.predict(eval_ds).predictions.astype(float)
pred.shape

array([[ 0.5546875 ],
       [ 0.68994141],
       [ 0.53369141],
       [ 0.31274414],
       [-0.01609802],
       [ 0.56689453],
       [ 0.53417969],
       [-0.02326965],
       [ 0.27514648],
       [ 1.0703125 ],
       [ 0.21069336],
       [ 0.27734375],
       [ 0.73779297],
       [ 0.86962891],
       [ 0.76171875],
       [ 0.56005859],
       [ 0.33618164],
       [ 0.01626587],
       [ 0.69726562],
       [ 0.37817383],
       [ 0.41870117],
       [ 0.22265625],
       [ 0.17773438],
       [ 0.24523926],
       [ 0.57666016],
       [-0.0383606 ],
       [-0.03845215],
       [-0.01753235],
       [-0.03546143],
       [ 0.61230469],
       [ 0.30175781],
       [ 0.01512909],
       [ 0.70898438],
       [ 0.54638672],
       [ 0.46679688],
       [ 0.26147461]])

In [ ]:
# Main takeaway
# input -> Transform(input) -> strings -> Tokenize(strings) -> numbers
# model(numbers) -> array of correlation (this is specific to this task)